# Homework 3
Complete the exercises working in your group. You may share the notebook with your group members using the share button in the upper right. 







# Question 1
## Plotting practice
These questions follow the examples presented in Notebook 9 from the Textbook. See that notebook for the dataset and template code.
*  Generate a violin plot that shows the distribution of height in each income group. Can you see a relationship between these variables?
* Make a boxplot that shows the distribution of weight in each income group. Plot the y-axis on a logarithmic scale.
* Generate a visualization of the relationship between weight and vegetable consumption. 

# Question 2
## Correlation
Recall that correlation only measures linear relationships. If the relationship is nonlinear, correlation generally underestimates how strong it is.

To demonstrate, generate another example of fake data that is different from the data presented in the text which was $y=x^2$. Explain what is the association between the variables (e.g. $y=x^4$, $y=sin(x)$) and why the correlation is low or zero despite there being a clear relationship between the variables.

# Question 3
## Simple regression
Who do you think eats more vegetables, people with low income, or people with high income? To answer this question, make a scatter plot with vegetable servings versus income, then estimate the slope of the relationship between vegetable consumption and income.

What is the slope of the regression line? Write a sentence that explains what this slope means in the context of the question we are exploring.

Finally, plot the regression line on top of the scatter plot.

# Question 4

## Inference
Complete the exercise from Notebook 11 which asks:
> But suppose we would like to know whether first babies are more unpredictable than other babies. In that case the test statistic we choose might be the standard deviation of pregnancy length, which is one way to quantify unpredictability.

1. Write a version of `test_stat` that computes the difference in standard deviation between the groups.

2. Write a loop that estimates the distribution of this test statistic under the null hypothesis.

3. Compute a p-value.

## Question 5
### AWS ML APIs
In the [lecture notebook](https://github.com/jeremyallenjacobson/qtm350/blob/master/CourseAssets/Rekognition_notebook/AWS-ML-API.ipynb) we walked you through setting up Sagemaker in the Educate account in order to use the AWS ML APIs.

Your task in this question is to select another ML service from AWS and then, using the documentation, create a walkthrough similar to the one we did above for the service you picked.

In your walkthrough, demonstrate how to call the service from within a Sagemaker notebook using either the AWS CLI or the Python SDK (your pick).

As always, be sure to narrate your code with lots of text, images, and links to share. Explain which ML APIs you were interested in, which you tried but couldn't use, and potential use cases that you would like to explore as well as data sources you would like to apply them to.

Finally, because we can't run Sagemaker code in a colab notebook, convert your walkthrough notebook from this exercise into a .html file using `jupyter nbconvert` as the command line. Then, create an S3 bucket, upload the .html file to your bucket, and follow the instructions [here](https://docs.aws.amazon.com/AmazonS3/latest/dev/HowDoIWebsiteConfiguration.html) for making the .html available as a webpage.

 Share the link to it in this notebook, so that I and other students can read your walkthrough.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=34182c1e-ced2-433c-9676-54ec654bbd88' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>